In [7]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()

In [8]:
# 进行随机下采样，并使用MLP分类
import scipy
import scipy.io as scio
import numpy as np
import os
# 读取训练和测试数据
data_root = '/data/file/classification_data/tmp/years/sum_2016'
trainX_norm = scio.loadmat(os.path.join(data_root, 'train_sumX_2016_norm.mat'))['X']
testX_norm = scio.loadmat(os.path.join(data_root, 'test_sumX_2016_norm.mat'))['X']

trainlabel = scio.loadmat(os.path.join(data_root, 'train_sumy_2016_label.mat'))['y']
testlabel = scio.loadmat(os.path.join(data_root, 'test_sumy_2016_label.mat'))['y']



In [9]:
X_re, y_re = enn.fit_resample(trainX_norm, trainlabel)
X_re
y_re = y_re.reshape(-1, 1)


In [10]:
# 从文档中，NU也是支撑载体分数的下限。 当我在具有20％正示例的培训集时出现这个问题时，
# 我必须从0.5到0.2的较低，或者将培训重叠设定为50％的正例。 提升nu不起作用
print((y_re == 1).sum() / len(y_re))  # 只有百分之一 
print(y_re.shape)

0.015522643121617772
(7022, 1)


In [11]:

# X_re = X_re[:5000]
# y_re = y_re[:5000]

In [12]:
X_re.shape, y_re.shape

((7022, 448), (7022, 1))

In [13]:
trainX_norm.shape

(7231, 448)

In [14]:
# scipy.io.savemat(os.path.join(data_root, 'SMOT/trainX.mat'), {'X': X_re})
# scipy.io.savemat(os.path.join(data_root, 'SMOT/trainy.mat'), {'X': y_re})



In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC
# nu 训练误差分数的上限和支撑载体分数的下限。 应该在间隔（0,1]。默认情况下0.5将被使用
clf = make_pipeline(StandardScaler(), NuSVC(nu=0.01))
# clf = NuSVC()
clf.fit(X_re, y_re)

/home/roczhang/anaconda3/envs/dataAna/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('nusvc', NuSVC(nu=0.01))])

In [22]:
clf.score(testX_norm, testlabel)

0.5089285714285714